In [1]:
# NYCbuswatcher fetch all of the shipments for a particular route and load them in as BusObservations

In [2]:
!pip install requests
import requests

In [3]:
import dateutil.parser
import json

class BusObservation():

    def to_serial(self):
        
        def serialize(obj):
            # Recursively walk object's hierarchy.
            
            if isinstance(obj, (bool, int, float)):
              return obj
            
            elif isinstance(obj, dict):
              obj = obj.copy()
              for key in obj:
                obj[key] = serialize(obj[key])
              return obj
            
            elif isinstance(obj, list):
              return [serialize(item) for item in obj]
            
            elif isinstance(obj, tuple):
              return tuple(serialize([item for item in obj]))
            
            elif hasattr(obj, '__dict__'):
              return serialize(obj.__dict__)
            
            else:
              # return repr(obj) # Don't know how to handle, convert to string
                return str(obj) # avoids single quotes around strings

        # return json.dumps(serialize(self))
        return serialize(self)

    def __repr__(self):
        output = ''
        # output = None
        for var, val in vars(self).items():
            if var == '_sa_instance_state':
                continue
            else:
                output = output + ('{} {} '.format(var,val))
        return output

    def __init__(self,route, shipment_bus):
        self.route = route
        for key in shipment_bus:
            setattr(self, key, shipment_bus[key])

In [4]:
route='M15'

In [5]:
base_url = 'http://api.buswatcher.org/api/v2/nyc'
shipments_for_route_url = '/{}'.format(route)

### load all Shipments for {route} into an array of BusObservations

In [6]:
buses = []

In [7]:
response = requests.get(base_url+shipments_for_route_url).json()

In [8]:
for s in response['shipments']:
    shipment = requests.get(s['url']).json()
    for bus_dict in shipment['buses']:
        buses.append(BusObservation(route, bus_dict))

In [9]:
len(buses)

20192

In [10]:
print(buses[12])

route M15 timestamp 2021-06-17 21:59:39-04:00 route_long MTA NYCT_M15 direction 1 service_date 2021-06-17 trip_id MTA NYCT_OH_B1-Weekday-SDon-121600_M15_256 gtfs_shape_id MTA_M150007 route_short M15 agency MTA NYCT origin_id MTA_401732 destination_name SOUTH FERRY via 2 AV lat 40.701549 lon -74.012199 bearing 196.69925 progress_rate noProgress progress_status layover vehicle_id MTA NYCT_5910 gtfs_block_id MTA NYCT_OH_B1-Weekday-SDon_E_OH_32700_M15-213 


### load all Shipments for {route} into a pandas dataframe

In [11]:
rows=[]

In [12]:
for s in response['shipments']:
    shipment = requests.get(s['url']).json()
    for bus_dict in shipment['buses']:
        rows.append(bus_dict)

In [13]:
!pip install pandas
import pandas as pd


In [19]:
df = pd.DataFrame.from_dict(rows, orient='columns')
df['passenger_count'] = df['passenger_count'].fillna(0)
df

,route,timestamp,route_long,direction,service_date,trip_id,gtfs_shape_id,route_short,agency,origin_id,...,next_stop_d,lat,lon,bearing,progress_rate,progress_status,vehicle_id,gtfs_block_id,next_stop_eta,passenger_count
0,M15,2021-06-17 21:58:34-04:00,MTA NYCT_M15,1,2021-06-17,MTA NYCT_OH_B1-Weekday-SDon-129200_M15_246,MTA_M150007,M15,MTA NYCT,MTA_401732,...,0.00,40.803182,-73.932486,157.238040,noProgress,layover,MTA NYCT_5901,MTA NYCT_OH_B1-Weekday-SDon_E_OH_23100_M15-217,NaN,0.0
1,M15,2021-06-17 21:58:43-04:00,MTA NYCT_M15,0,2021-06-17,MTA NYCT_OH_B1-Weekday-SDon-127900_M15_250,MTA_M150004,M15,MTA NYCT,MTA_803019,...,9.93,40.740499,-73.975897,53.660603,normalProgress,NaN,MTA NYCT_6035,MTA NYCT_OH_B1-Weekday-SDon_E_OH_21420_M15-214,2021-06-17T21:58:52.354-04:00,10.0
2,M15,2021-06-17 21:58:38-04:00,MTA NYCT_M15,1,2021-06-17,MTA NYCT_OH_B1-Weekday-SDon-121600_M15_256,MTA_M150007,M15,MTA NYCT,MTA_401732,...,NaN,40.701549,-74.012199,196.699250,noProgress,layover,MTA NYCT_5910,MTA NYCT_OH_B1-Weekday-SDon_E_OH_32700_M15-213,NaN,0.0
3,M15,2021-06-17 21:58:22-04:00,MTA NYCT_M15,0,2021-06-17,MTA NYCT_OH_B1-Weekday-SDon-124500_M15_239,MTA_M150005,M15,MTA NYCT,MTA_405359,...,12.75,40.803046,-73.932167,157.094220,noProgress,layover,MTA NYCT_5886,MTA NYCT_OH_B1-Weekday-SDon_E_OH_28620_M15-228,2021-06-17T21:58:52.354-04:00,0.0
4,M15,2021-06-17 21:58:41-04:00,MTA NYCT_M15,1,2021-06-17,MTA NYCT_OH_B1-Weekday-SDon-122600_M15_245,MTA_M150007,M15,MTA NYCT,MTA_401732,...,24.10,40.701608,-74.011920,191.821490,noProgress,layover,MTA NYCT_5912,MTA NYCT_OH_B1-Weekday-SDon_E_OH_24480_M15-219,2021-06-17T21:58:52.354-04:00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20187,M15,2021-06-19 11:10:11-04:00,MTA NYCT_M15,1,2021-06-19,MTA NYCT_OH_B1-Saturday-067200_M15_227,MTA_M150007,M15,MTA NYCT,MTA_401732,...,169.57,40.803231,-73.932603,157.238040,noProgress,layover,MTA NYCT_5839,MTA NYCT_OH_B1-Saturday_A_OH_19620_M15-211,2021-06-19T11:10:44.664-04:00,0.0
20188,M15,2021-06-19 11:10:26-04:00,MTA NYCT_M15,1,2021-06-19,MTA NYCT_OH_B1-Saturday-063200_M15_215,MTA_M150007,M15,MTA NYCT,MTA_401732,...,208.62,40.774538,-73.954171,234.293300,normalProgress,NaN,MTA NYCT_5836,MTA NYCT_OH_B1-Saturday_A_OH_27240_M15-215,2021-06-19T11:11:28.905-04:00,0.0
20189,M15,2021-06-19 11:10:34-04:00,MTA NYCT_M15,0,2021-06-19,MTA NYCT_OH_B1-Saturday-065800_M15_206,MTA_M150005,M15,MTA NYCT,MTA_405359,...,212.65,40.727990,-73.985019,53.880660,normalProgress,NaN,MTA NYCT_6053,MTA NYCT_OH_B1-Saturday_A_OH_14760_M15-208,2021-06-19T11:11:50.964-04:00,11.0
20190,M15,2021-06-19 11:10:33-04:00,MTA NYCT_M15,1,2021-06-19,MTA NYCT_OH_B1-Saturday-059600_M15_214,MTA_M150008,M15,MTA NYCT,MTA_401732,...,250.86,40.711466,-73.992397,278.325650,normalProgress,NaN,MTA NYCT_6065,MTA NYCT_OH_B1-Saturday_A_OH_25440_M15-214,2021-06-19T11:16:12.963-04:00,4.0
